In [1]:
# Imports
#---------
import sys
import pandas as pd
import numpy as np
import feather
import os
import gc
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
import matplotlib.ticker as ticker
from matplotlib.dates import DateFormatter
import matplotlib as mpl
from datetime import timezone

register_matplotlib_converters()

# File locations
#----------------
the_gases = './oxaria/data/raw/2oxaria/json/gap_filling/'
pngs = './oxaria/data/pngs/gap_filling/'


In [2]:
# Load the df 
#-------------
oxaria2_climate = pd.read_feather(the_gases+'oxaria2_climate_gf.ftr').set_index(['tag','rec'])
oxaria2_climate_lt = oxaria2_climate.iloc[:,[0,1,3]]


In [3]:
# Checking out the data types 
#-----------------------------
print('\n'+'Loading Oxaria2 climate...\n')
oxaria2_climate_lt.info()



Loading Oxaria2 climate...

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4137653 entries, ('scs-bgx-550', Timestamp('2020-03-09 10:31:40+0000', tz='UTC')) to ('scs-bgx-559', Timestamp('2021-02-28 23:59:30+0000', tz='UTC'))
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   val.hmd  float32
 1   val.tmp  float32
 2   name     object 
dtypes: float32(2), object(1)
memory usage: 269.5+ MB


In [4]:
# Some cleaning incase have missed anything
#-------------------------------------------
oxaria2_climate_lt = oxaria2_climate_lt[~oxaria2_climate.index.duplicated(keep='last')]
oxaria2_climate_lt = oxaria2_climate_lt.reset_index()
oxaria2_climate = oxaria2_climate_lt.dropna(axis=0,subset=['tag']).set_index(['tag','rec'])
oxaria2_climate_lt.sort_index()
print('\n Cleaning duplicates & NAs in index (if any)...\n')


 Cleaning duplicates & NAs in index (if any)...



In [5]:
# Get a list of unique tags
#---------------------------
tags = oxaria2_climate.index.get_level_values(0).unique()
print('\n Unique tags loaded...\n \n', tags)



 Unique tags loaded...
 
 Index(['scs-bgx-550', 'scs-bgx-551', 'scs-bgx-552', 'scs-bgx-553',
       'scs-bgx-554', 'scs-bgx-555', 'scs-bgx-556', 'scs-bgx-557',
       'scs-bgx-558', 'scs-bgx-559'],
      dtype='object', name='tag')


In [6]:
# Define the start dates for stable operation
#---------------------------------------------
start_dates = ['2020-12-15T00:00:00','2020-07-20T00:00:00','2020-12-15T00:00:00','2020-07-01T00:00:00', \
               '2020-07-01T00:00:00','2020-12-15T00:00:00','2020-07-20T00:00:00','2020-07-01T00:00:00', \
               '2020-12-15T00:00:00','2020-12-15T00:00:00']

dates_list = [dt.datetime.fromisoformat(date).replace(tzinfo=timezone.utc) for date in start_dates]
dates_dict = dict(zip(tags,dates_list))
print('\n Applying variable start date filters of each sensor...\n \n'+str(dates_dict))


 Applying variable start date filters of each sensor...
 
{'scs-bgx-550': datetime.datetime(2020, 12, 15, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-551': datetime.datetime(2020, 7, 20, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-552': datetime.datetime(2020, 12, 15, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-553': datetime.datetime(2020, 7, 1, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-554': datetime.datetime(2020, 7, 1, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-555': datetime.datetime(2020, 12, 15, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-556': datetime.datetime(2020, 7, 20, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-557': datetime.datetime(2020, 7, 1, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-558': datetime.datetime(2020, 12, 15, 0, 0, tzinfo=datetime.timezone.utc), 'scs-bgx-559': datetime.datetime(2020, 12, 15, 0, 0, tzinfo=datetime.timezone.utc)}


In [9]:
# Select periods of stable operation from the df of all gases sesnor data
#-------------------------------------------------------------------------
print('\n Applying start date filters...\n')

tmp = []

for k,v in dates_dict.items():
    df = oxaria2_climate_lt.query('tag == @k & rec >= @v')
    tmp.append(df)
tmpdf = pd.concat(tmp)
print(tmpdf.info())

# Save the df 
#-------------
print('\n Writing to  '+the_gases+'oxaria2_climate_stable_356_2feb21/.ftr\n')
tmpdf.reset_index().to_feather(the_gases+'oxaria2_climate_stable_356_2feb21/.ftr')
print('All done! \U0001F600')


 Applying start date filters...

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000303 entries, 353407 to 4137652
Data columns (total 5 columns):
 #   Column   Dtype              
---  ------   -----              
 0   tag      object             
 1   rec      datetime64[ns, UTC]
 2   val.hmd  float32            
 3   val.tmp  float32            
 4   name     object             
dtypes: datetime64[ns, UTC](1), float32(2), object(2)
memory usage: 76.3+ MB
None

 Writing to  ./oxaria/data/raw/2oxaria/json/gap_filling/oxaria2_climate_stable_356_2feb21/.ftr

All done! 😀
